In [5]:
# import libraries
#!pip install BeautifulSoup4
#!pip install urllib3
#import urllib
#import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
from datetime import date 

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Pass cookies to be used to authenticate logon at site

In [159]:
df_cookies = pd.read_csv(r'C:\Users\stefa\Documents\GitHub\first_scraper-master\cookies.csv',header=None).dropna()
names = list(df_cookies[0])
values = list(df_cookies[1])
cookies = dict(zip(names, values))
#cookies
#df_cookies.set_index(df_cookies[0])#,drop=True, inplace=True)#.to_dict()

### List of addresses

In [169]:
# Add count to end URL, 0 for first 25, 25 for next 25, 50... 
season_avg_url = 'https://basketball.fantasysports.yahoo.com/nba/14392/players?status=ALL&pos=P&cut_type=33&stat1=S_AS_2018&myteam=0&sort=AR&sdir=1&count='
day30_avg_url = 'https://basketball.fantasysports.yahoo.com/nba/14392/players?status=ALL&pos=P&cut_type=33&stat1=S_AL30&myteam=0&sort=AR&sdir=1&count='
day14_avg_url = 'https://basketball.fantasysports.yahoo.com/nba/14392/players?status=ALL&pos=P&cut_type=33&stat1=S_AL14&myteam=0&sort=AR&sdir=1&count='


### Pull function
- Pulls based on specified URL request
- Cookies used to establish logon session from cookies.csv
- Creates player array for 25 players at a time
- Loop to create full datasets

In [204]:
def pull_25(url,ranks):
    r = requests.post(str(url+ranks), cookies=cookies)
    soup = BeautifulSoup(r.text,'lxml')
    table = soup.find('table',{'class':'Table Ta-start Fz-xs Table-mid Table-px-xs Table-interactive'})
    mydivs = table.findAll('div', {"class": "ysf-player-name Nowrap Grid-u Relative Lh-xs Ta-start"})
    player_list = []
    for player in mydivs:
        player_list.append(player.text)
    df_player = pd.DataFrame(player_list)   
    dummy_array = np.empty((27,23))
    dummy_array[:] = np.nan
    player_array = pd.DataFrame(dummy_array)
    row_marker=0
    for row in table.findAll('tr'):
        column_marker = 0
        for data in row.findAll('td'):
            player_array.iloc[row_marker,column_marker] = data.get_text()
            column_marker += 1
        row_marker+=1
    player_array['INJ'] = np.where(player_array[1].str.contains("INJ"),"INJ",0)
    player_array = player_array.dropna()
    columns_we_like = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,'INJ']
    player_array = player_array[columns_we_like].reset_index(drop=True)
    df_player = df_player.merge(player_array, left_index =True, right_index=True)
    df_player.columns=['Player','Owner','GP','O_Rank','C_Rank','Owned','MP','FGs','FG_pct','FTs','FT_pct',
                       '3pt','Pts','Rbs','Ast','Stl','Blk','TO','INJ']
    return df_player

### Build top 200

In [213]:
df_player = pull_25(day30_avg_url,str(0))
for x in range(25,200,25):
    df_temp = pull_25(day30_avg_url,str(x))
    df_player = df_player.append(df_temp)
    time.sleep(.25) 
df_player = df_player.reset_index(drop=True)

In [230]:
path = r'C:\Users\stefa\Documents\GitHub\first_scraper-master\Data'
today = str(date.today())

df_player.to_csv(path+"_"+today+'_avg_30.csv',encoding='utf-8')
